# 06: Prompt Engineering for Better BS Detection

Learn how to craft effective prompts for more accurate BS detection.

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from config.llm_factory import LLMFactory
from pydantic import BaseModel

llm = LLMFactory.create_llm()
print("✅ LLM ready for prompt engineering!")

✅ LLM ready for prompt engineering!


## 1. Basic vs. Structured Prompts

In [3]:
claim = "The Boeing 747 can fly in reverse using reverse thrust"

# Basic prompt
basic_prompt = f"Is this true: {claim}"
basic_response = llm.invoke(basic_prompt)
print("Basic Prompt Response:")
print(basic_response.content[:200] + "...\n")

# Structured prompt
structured_prompt = f"""You are an aviation expert.

Analyze this claim: {claim}

Provide:
- Verdict: [TRUE/FALSE]
- Confidence: [0-100%]
- Evidence: 2 key facts
"""

structured_response = llm.invoke(structured_prompt)
print("\nStructured Prompt Response:")
print(structured_response.content)

Basic Prompt Response:
No, the Boeing 747 cannot fly in reverse using reverse thrust.

While the Boeing 747—and many other jet aircraft—are equipped with thrust reversers, these are designed to help slow the aircraft down a...


Structured Prompt Response:
- Verdict: FALSE  
- Confidence: 95%  
- Evidence:  
  1. Reverse thrust on the Boeing 747 is designed to slow the aircraft down after landing by redirecting engine thrust forward, not to propel the aircraft backward in flight or on the ground.  
  2. The 747 does not have any mechanism, such as reversible pitch propellers or dedicated reverse propulsion systems, that would allow it to taxi or fly in reverse; ground movement in reverse is typically achieved by using a pushback tug.


## 2. Using Pydantic for Structured Output

In [4]:
class BSDetectionResult(BaseModel):
    verdict: str  # "BS" or "LEGITIMATE"
    confidence: int  # 0-100
    reasoning: str
    evidence: list[str]

# Bind the model to get structured output
llm_with_structure = llm.with_structured_output(BSDetectionResult)

prompt = f"""Analyze if this aviation claim is BS or legitimate: {claim}

Consider physics, engineering constraints, and known aircraft capabilities."""

result = llm_with_structure.invoke(prompt)
print(f"Verdict: {result.verdict}")
print(f"Confidence: {result.confidence}%")
print(f"Reasoning: {result.reasoning}")
print(f"Evidence: {result.evidence}")

Verdict: BS
Confidence: 95%
Reasoning: While the Boeing 747 is equipped with thrust reversers that redirect engine thrust forward to aid in deceleration during landing, these thrust reversers cannot produce sustained reverse thrust sufficient to propel the aircraft backward in flight or on the ground under its own power. Physically and aerodynamically, the engines are designed to push air backward to generate forward thrust; the thrust reversers redirect the airflow forward but primarily to slow the aircraft and not to move it backward. Additionally, operational procedures and engineering constraints prevent the aircraft from flying or taxiing in reverse using engine thrust. Ground maneuvering in reverse is typically performed using tugs, and in flight, reverse thrust is not used to reverse direction. Therefore, the claim that the Boeing 747 can fly in reverse using reverse thrust is false.
Evidence: ['Boeing 747 engine thrust reversers are designed to help decelerate the aircraft upon

## 3. Few-Shot Learning

In [5]:
few_shot_prompt = """Determine if aviation claims are BS or LEGITIMATE.

Examples:
Claim: "The Wright brothers first flew in 1903"
Verdict: LEGITIMATE (Historical fact)

Claim: "Helicopters fly by pushing air down"
Verdict: LEGITIMATE (Correct physics)

Claim: "Jets can hover vertically without special engines"
Verdict: BS (Requires VTOL capability)

Now analyze:
Claim: "Commercial planes fly at the speed of sound"
Verdict:"""

response = llm.invoke(few_shot_prompt)
print(response.content)

Claim: "Commercial planes fly at the speed of sound"  
Verdict: BS (Most commercial planes fly below the speed of sound; only specialized aircraft like the Concorde flew at supersonic speeds)


## 4. Chain of Thought Prompting

In [7]:
cot_prompt = f"""Think step-by-step to determine if this claim is BS:

Claim: "{claim}"

Step 1: What does the claim assert?
Step 2: What physics/engineering principles apply?
Step 3: Is it technically possible?
Step 4: Final verdict

Let's work through each step:"""

response = llm.invoke(cot_prompt)
print(response.content)

Sure! Let’s analyze the claim step-by-step.

---

**Step 1: What does the claim assert?**  
The claim says: *"The Boeing 747 can fly in reverse using reverse thrust."*  
This means the airplane, a large commercial jet, can move backward through the air (fly backwards) by using its engines' reverse thrust capability.

---

**Step 2: What physics/engineering principles apply?**  
- **Thrust and Aerodynamics:** Forward flight occurs when the engines produce thrust forward, overcoming drag and allowing lift to keep the plane airborne.  
- **Reverse Thrust:** Many jet engines have a reverse thrust mechanism—usually deployed after landing—to help slow the aircraft by redirecting engine thrust forward, opposing the forward motion on the ground.  
- **Flight Dynamics:** To fly, the aircraft must generate lift through airflow over wings. The wings are designed for forward airflow; flying backward would drastically reduce lift and control.  
- **Control Surfaces:** Elevators, rudders, and ailero

## 5. Prompt Template for Production

In [8]:
def create_bs_detection_prompt(claim: str, domain: str = "general") -> str:
    """Create an optimized BS detection prompt."""
    return f"""You are an expert fact-checker specializing in {domain}.

Analyze this claim for accuracy: "{claim}"

Consider:
1. Known facts and data
2. Physical/technical feasibility
3. Logical consistency
4. Common misconceptions

Provide a structured analysis with verdict, confidence, and evidence."""

# Test with different domains
test_claims = [
    ("AI will achieve consciousness by 2025", "technology"),
    ("The moon landing was filmed in a studio", "history"),
]

for claim, domain in test_claims:
    print(f"\n{'='*50}")
    print(f"Claim: {claim}")
    prompt = create_bs_detection_prompt(claim, domain)
    result = llm_with_structure.invoke(prompt)
    print(f"Verdict: {result.verdict} ({result.confidence}%)")


Claim: AI will achieve consciousness by 2025
Verdict: False (85%)

Claim: The moon landing was filmed in a studio
Verdict: False (95%)


## Key Takeaways

1. **Structure beats length** - Clear format > verbose instructions
2. **Pydantic ensures consistency** - Structured output every time
3. **Examples improve accuracy** - Few-shot > zero-shot
4. **Chain of thought helps complex claims** - Step-by-step reasoning
5. **Domain-specific prompts work better** - Tailor to your use case

Next: Combine these techniques with our multi-agent system!